---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

!pip install xlrd

    100% |████████████████████████████████| 112kB 2.6MB/s 


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
from google.colab import drive
drive.mount('/data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /data


In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def normalize(text):
    return text.strip()
  
def read_university_towns(path_file="university_towns.txt"):
    ct = pd.read_csv(path_file, sep="\t", header=None, names=["all"])

    university_towns = pd.DataFrame(columns=["University", "RegionName", "State"])

    for index, row in ct.iterrows():
        name = row["all"]

        if "[edit]" in name:
            state = normalize(name[0:name.find("[")])
        else:
            index_towns = name.find("(")

            if index_towns <= 0:
                index_towns = name.find(",")

            if index_towns <= 0:
                print(name)
            else:
                towns = normalize(name[:index_towns])
                universities = name[index_towns + 1:]

                str_split = universities.split(",")

                for university in str_split:
                    df = pd.DataFrame({'University': [normalize(university)], 'RegionName': [towns], 'State': [state]})
                    university_towns = university_towns.append(df, ignore_index=True)

    return university_towns

def get_list_of_university_towns():
    ut = read_university_towns("/data/My Drive/DS1/data/university_towns.txt").drop(["University"], axis=1, index=None).drop_duplicates(keep='first').reset_index(drop=True)
    return ut
  
get_list_of_university_towns()

The Colleges of Worcester Consortium:
The Five College Region of Western Massachusetts:


,RegionName,State
0,Auburn,Alabama
1,Florence,Alabama
2,Jacksonville,Alabama
3,Livingston,Alabama
4,Montevallo,Alabama
5,Troy,Alabama
6,Tuscaloosa,Alabama
7,Tuskegee,Alabama
8,Fairbanks,Alaska
9,Flagstaff,Arizona


In [5]:
def read_gdp(path_file="gdplev.xls"):
    gdp = pd.read_excel(path_file, skiprows=range(7), header=0)
    gdp.drop(columns=["Unnamed: 0", "Unnamed: 1", "Unnamed: 2", "Unnamed: 3", "Unnamed: 7"], inplace=True)
    gdp.columns = ["year_quarter", "current", "2009"]
    gdp["year"] = gdp.year_quarter.str[0:4].astype(int)
    gdp = gdp[gdp.year >= 2000]

    return gdp


states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


def convert_state(state):
    if state in states:
        return states[state]

def get_recession_start():
    gdp = read_gdp("/data/My Drive/DS1/data/gdplev.xls")
    for i in range(1, len(gdp) - 1):
        if gdp.iloc[i - 1, 2] > gdp.iloc[i, 2] > gdp.iloc[i + 1, 2]:
            return gdp.iloc[i - 1, 0]

    return None

get_recession_start()

'2008q2'

In [6]:
def get_recession_end():
    gdp = read_gdp("/data/My Drive/DS1/data/gdplev.xls")
    is_not_stated = True

    for i in range(1, len(gdp) - 1):
        if is_not_stated:
            if gdp.iloc[i - 1, 2] > gdp.iloc[i, 2] > gdp.iloc[i + 1, 2]:
                is_not_stated = False
        else:
            if gdp.iloc[i - 1, 2] < gdp.iloc[i, 2] < gdp.iloc[i + 1, 2]:
                return gdp.iloc[i + 1, 0]

    return None
  
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    gdp = read_gdp("/data/My Drive/DS1/data/gdplev.xls")
    for i in range(1, len(gdp) - 1):
        if gdp.iloc[i - 1, 2] > gdp.iloc[i, 2] > gdp.iloc[i + 1, 2]:
           return gdp.iloc[i + 1, 0]
    
    return None

get_recession_bottom()

'2008q4'

In [8]:
def read_home(path_file="City_Zhvi_AllHomes.csv"):
    home = pd.read_csv(path_file)

    home["State"] = home["State"].map(lambda x: convert_state(x))

    columns = ["RegionName", "State", "SizeRank"]
    for x in range(2000, 2017):
        for y in range(1, 5):
            if x > 2015 & y > 3:
                break

            column = str(x) + "q" + str(y)

            t1 = str(x) + "-" + str((y - 1) * 3 + 1).zfill(2)
            home[t1] = pd.to_numeric(home[t1], errors='coerce')

            t2 = str(x) + "-" + str((y - 1) * 3 + 2).zfill(2)
            home[t2] = pd.to_numeric(home[t2], errors='coerce')

            if x > 2015 & y > 2:
                home[column] = home[[t1, t2]].mean(axis=1)
            else:
                t3 = str(x) + "-" + str((y - 1) * 3 + 3).zfill(2)
                home[t3] = pd.to_numeric(home[t3], errors='coerce')
                home[column] = home[[t1, t2, t3]].mean(axis=1)

            columns.append(column)

    home = home[columns]

    return home
  
def convert_housing_data_to_quarters():
    home = read_home("/data/My Drive/DS1/data/City_Zhvi_AllHomes.csv")
    home = home.drop("SizeRank", axis=1)
    home = home.set_index(["RegionName", "State"], drop=True)

    return home

  
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2001q1,2001q2,2001q3,2002q1,2002q2,2002q3,2003q1,...,2013q3,2014q1,2014q2,2014q3,2015q1,2015q2,2015q3,2016q1,2016q2,2016q3
RegionName,State,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,479500.0,5.031333e+05,5.154667e+05,522300.0,5.322667e+05,5.408000e+05,554300.0,5.828667e+05,5.916333e+05,587200.0
Los Angeles,California,2.070667e+05,2.144667e+05,220050.0,2.330000e+05,2.391000e+05,243950.0,2.619667e+05,2.727000e+05,284400.0,3.189000e+05,...,451650.0,4.842667e+05,4.980333e+05,507300.0,5.288000e+05,5.381667e+05,545600.0,5.660333e+05,5.774667e+05,584050.0
Chicago,Illinois,1.384000e+05,1.436333e+05,147300.0,1.569333e+05,1.618000e+05,165850.0,1.755000e+05,1.775667e+05,178250.0,1.871000e+05,...,179600.0,1.909000e+05,1.926333e+05,194750.0,2.010667e+05,2.060333e+05,207900.0,2.060667e+05,2.082000e+05,212000.0
Philadelphia,Pennsylvania,5.300000e+04,5.363333e+04,53950.0,5.533333e+04,5.553333e+04,56100.0,5.913333e+04,6.073333e+04,62000.0,6.473333e+04,...,110950.0,1.137333e+05,1.137333e+05,115200.0,1.162000e+05,1.179667e+05,120700.0,1.234333e+05,1.269333e+05,128700.0
Phoenix,Arizona,1.118333e+05,1.143667e+05,115750.0,1.196000e+05,1.215667e+05,122550.0,1.265333e+05,1.283667e+05,130000.0,1.357000e+05,...,152050.0,1.632333e+05,1.642667e+05,164950.0,1.715333e+05,1.741667e+05,178100.0,1.879000e+05,1.914333e+05,195200.0
Las Vegas,Nevada,1.326000e+05,1.343667e+05,135100.0,1.395333e+05,1.417333e+05,142950.0,1.493333e+05,1.509333e+05,152000.0,1.594667e+05,...,152800.0,1.660000e+05,1.700667e+05,173000.0,1.775000e+05,1.816000e+05,186250.0,1.946000e+05,1.972000e+05,199950.0
San Diego,California,2.229000e+05,2.343667e+05,243650.0,2.672000e+05,2.762667e+05,283250.0,3.012333e+05,3.128667e+05,327900.0,3.664333e+05,...,451950.0,4.740000e+05,4.802000e+05,487450.0,5.033667e+05,5.120667e+05,518750.0,5.293333e+05,5.362333e+05,539750.0
Dallas,Texas,8.446667e+04,8.386667e+04,84700.0,8.973333e+04,8.930000e+04,89000.0,9.256667e+04,9.380000e+04,95100.0,9.820000e+04,...,101850.0,1.053333e+05,1.066333e+05,108450.0,1.137000e+05,1.211333e+05,127500.0,1.405000e+05,1.446000e+05,149300.0
San Jose,California,3.742667e+05,4.065667e+05,428200.0,4.706667e+05,4.702000e+05,458550.0,4.414333e+05,4.577667e+05,471850.0,4.782667e+05,...,630850.0,6.611667e+05,6.794000e+05,693700.0,7.314333e+05,7.567333e+05,774200.0,8.036000e+05,8.189333e+05,822200.0


In [11]:
def is_university_town(town, university_towns):

    if town in university_towns:
        return True
    else:
        return False


def price_ratio(price_home):
    return (price_home['2008q3'] - price_home['2009q2']) / price_home['2008q3']


def check_mean(not_in_university_town, in_university_town):
    if not_in_university_town.mean() < in_university_town.mean():
        return 'non-university town'
    else:
        return 'university town'


def run_ttest():
    home = convert_housing_data_to_quarters()
    home = home.loc[:, '2008q3': '2009q2']

    home = home.reset_index()

    home['up&down'] = home.apply(price_ratio, axis=1)

    university_towns = get_list_of_university_towns()

    university_towns = set(university_towns["RegionName"])

    home['is_university_town'] = home.apply(lambda row: is_university_town(row["RegionName"], university_towns), axis=1)

    not_in_university_town = home[home['is_university_town'] == 0].loc[:, 'up&down'].dropna()
    in_university_town = home[home['is_university_town'] == 1].loc[:, 'up&down'].dropna()

    p_val = ttest_ind(not_in_university_town, in_university_town)[1]

    return p_val < 0.01, p_val, check_mean(not_in_university_town, in_university_town)
  
run_ttest()

The Colleges of Worcester Consortium:
The Five College Region of Western Massachusetts:


(True, 0.0002556257247981708, 'university town')